livree par client employee et date meme chose avec non livreee

# Livree

## Command + client

# Resultat + employee

# non Livree

# model pour represente un entropot projet te3na ndiro etoile

Etoile / constelation / flocan de naige 
Table de fait / Table de dimention
1/etile ( une seul table de fait central - pas de relation entre les tables de dimentions mais il ya entre fait et dimention)
pas de relation entre les dimentions alors abscence de hierachie entre les dim

etoile (simple) ---> flocan (une seul table de fait +dimention seoare+heiarchi entre dim +normalisation) ---> constelation(redondance plusieur fait dim commune entre fait+hearchie entre dimention) ( koul wa7ed rigla lo5er)

mesure = client , ordre , employee

dim (axe d'analise) = client , ordre , employee,temps (lazem dire dim temps )

multi dimention = donne de plusieur tables

Fait te3na hya command 3endha id howa id sequenciel ( num qui incremante) + concatination key primaire des dimentions id1id2...
+ attribute te3 command nbr commandes livree/non livree

# Cleaning ACCESS

- **Customers**

In [ ]:
for i in [customers,order_details,orders,products,purchase_orders,shippers,suppliers]:
        i.drop_duplicates(inplace=True)

#['Order Date'] = i['Order Date'].str.strip()

customers['ContactName'] = (
    customers['First Name'].fillna('') + ' ' + customers['Last Name'].fillna('')
)
customers['ContactName'] = customers['ContactName'].str.strip().str.replace(r'\s+', ' ', regex=True)
customers.drop(columns=['First Name', 'Last Name'], inplace=True)

customers.rename(columns={'Country/Region':'Country'},inplace=True)
customers.drop(columns=['E-mail Address','Job Title','Business Phone','Home Phone','Mobile Phone',
                        'Fax Number','Address','City','State/Province','ZIP/Postal Code','Web Page',
                        'Notes','Attachments'],inplace=True)

customers.to_csv('customer.csv', index=False)

- **orders**

In [ ]:
orders['Order Date'] = pd.to_datetime(orders['Order Date']).dt.strftime('%Y-%m-%d')
orders['Shipped Date'] = pd.to_datetime(orders['Shipped Date']).dt.strftime('%Y-%m-%d')

cols_to_drop = [
    'Ship Name', 'Ship Address', 'Ship City', 'Ship State/Province',
    'Ship ZIP/Postal Code', 'Ship Country/Region',
    'Taxes', 'Payment Type', 'Paid Date',
    'Notes', 'Tax Rate', 'Tax Status', 'Status ID'
]
orders.drop(columns=[c for c in cols_to_drop if c in orders.columns], inplace=True)




orders.columns = orders.columns.str.replace(' ','',regex = False)

orders.to_csv('order.csv', index=False)


- **Employee**

In [ ]:
if 'First Name' in employees.columns or 'Last Name' in employees.columns:
    employees['FullName'] = (
        employees.get('First Name', '').fillna('') + ' ' + employees.get('Last Name', '').fillna('')
    ).str.strip()
    # drop the original name columns if they exist
    employees.drop(columns=[c for c in ['First Name', 'Last Name'] if c in employees.columns], inplace=True)

cols_to_remove = [
    'E-mail Address','Job Title','Business Phone','Home Phone','Mobile Phone',
    'Fax Number','Address','City','State/Province','ZIP/Postal Code',
    'Country/Region','Web Page','Notes','Attachments'
]
employees.drop(columns=[c for c in cols_to_remove if c in employees.columns], inplace=True)

employees.to_csv('employees.csv', index=False)

# Cleaning SQL

In [ ]:
df_customers.rename(columns={'CompanyName':'Company'},inplace=True)
cols_to_drop = [
    'ContactTitle','Address','City','Region','PostalCode','Phone','Fax'
]
df_customers.drop(columns=[c for c in cols_to_drop if c in df_customers.columns], inplace=True)


# ANALYTICS

## 1- TRANSFORM WAREHOUSE INTO CSV FILES

In [7]:
import pandas as pd
from sqlalchemy import create_engine
import urllib

# Encode the ODBC driver name
params = urllib.parse.quote_plus(
    "Driver={ODBC Driver 17 for SQL Server};"
    "Server=localhost\\SQLEXPRESS;"
    "Database=Northwind_DW;"
    "Trusted_Connection=yes;"
)

engine = create_engine(f"mssql+pyodbc:///?odbc_connect={params}")


dim_customer = pd.read_sql("SELECT * FROM DimCustomer", engine)
dim_employee = pd.read_sql("SELECT * FROM DimEmployee", engine)
dim_time = pd.read_sql("SELECT * FROM DimTime", engine)
order_fact = pd.read_sql("SELECT * FROM Order_Fact", engine)

# Export to CSV
dim_customer.to_csv("../Data/warehouse/DimCustomer.csv", index=False)
dim_employee.to_csv("../Data/warehouse/DimEmployee.csv", index=False)
dim_time.to_csv("../Data/warehouse/DimTime.csv", index=False)
order_fact.to_csv("../Data/warehouse/Order_Fact.csv", index=False)


OperationalError: (pyodbc.OperationalError) ('08001', '[08001] [Microsoft][ODBC Driver 17 for SQL Server]SQL Server Network Interfaces: Error Locating Server/Instance Specified [xFFFFFFFF].  (-1) (SQLDriverConnect); [08001] [Microsoft][ODBC Driver 17 for SQL Server]Login timeout expired (0); [08001] [Microsoft][ODBC Driver 17 for SQL Server]A network-related or instance-specific error has occurred while establishing a connection to SQL Server. Server is not found or not accessible. Check if instance name is correct and if SQL Server is configured to allow remote connections. For more information see SQL Server Books Online. (-1)')
(Background on this error at: https://sqlalche.me/e/20/e3q8)

## 2- TRANSFORM THE WAREHOUSE CSV FILES INTO SPECIFIC ANALYTICS / DASHBOARDS

In [1]:

import pandas as pd

customers = pd.read_csv("../Data/warehouse/DimCustomer.csv")
employees = pd.read_csv("../Data/warehouse/DimEmployee.csv")
orders = pd.read_csv("../Data/warehouse/Order_Fact.csv")
time_dim = pd.read_csv("../Data/warehouse/DimTime.csv")

customers.columns = customers.columns.str.strip()
employees.columns = employees.columns.str.strip()
orders.columns = orders.columns.str.strip()
time_dim.columns = time_dim.columns.str.strip()

orders['ShippedDateKey'] = orders['ShippedDateKey'].astype(int)

shipped_summary = pd.DataFrame({
    'ShippedStatus': ['Shipped', 'NotShipped'],
    'Count': [
        (orders['ShippedDateKey'] != 1011900).sum(),
        (orders['ShippedDateKey'] == 1011900).sum()
    ]
})
shipped_summary.to_csv("../Data/analytics/shipped_summary.csv", index=False)



shipped_by_employee = orders[orders['ShippedDateKey'] != 1011900]\
    .groupby('EmployeeID').size().reset_index(name='ShippedCount')
shipped_by_employee = shipped_by_employee.merge(employees[['EmployeeID','FullName']], on='EmployeeID')
shipped_by_employee.to_csv("../Data/analytics/shipped_by_employee.csv", index=False)


shipped_by_customer = orders[orders['ShippedDateKey'] != 1011900]\
    .groupby('CustomerID').size().reset_index(name='ShippedCount')
shipped_by_customer = shipped_by_customer.merge(customers[['CustomerID','Company']], on='CustomerID')
shipped_by_customer.to_csv("../Data/analytics/shipped_by_customer.csv", index=False)

shipped_by_country = orders[orders['ShippedDateKey'] != 1011900]\
    .merge(customers[['CustomerID','Country']], on='CustomerID')\
    .groupby('Country').size().reset_index(name='ShippedCount')
shipped_by_country.to_csv("../Data/analytics/shipped_by_country.csv", index=False)

shipped_by_company = orders[orders['ShippedDateKey'] != 1011900]\
    .merge(customers[['CustomerID','Company']], on='CustomerID')\
    .groupby('Company').size().reset_index(name='ShippedCount')
shipped_by_company.to_csv("../Data/analytics/shipped_by_company.csv", index=False)



# Not shipped by employee
not_shipped_by_employee = orders[orders['ShippedDateKey'] == 1011900]\
    .groupby('EmployeeID').size().reset_index(name='NotShippedCount')
not_shipped_by_employee = not_shipped_by_employee.merge(employees[['EmployeeID','FullName']], on='EmployeeID')
not_shipped_by_employee.to_csv("../Data/analytics/not_shipped_by_employee.csv", index=False)

# Not shipped by customer
not_shipped_by_customer = orders[orders['ShippedDateKey'] == 1011900]\
    .groupby('CustomerID').size().reset_index(name='NotShippedCount')
not_shipped_by_customer = not_shipped_by_customer.merge(customers[['CustomerID','Company']], on='CustomerID')
not_shipped_by_customer.to_csv("../Data/analytics/not_shipped_by_customer.csv", index=False)

# Not shipped by country
not_shipped_by_country = orders[orders['ShippedDateKey'] == 1011900]\
    .merge(customers[['CustomerID','Country']], on='CustomerID')\
    .groupby('Country').size().reset_index(name='NotShippedCount')
not_shipped_by_country.to_csv("../Data/analytics/not_shipped_by_country.csv", index=False)

# Not shipped by company
not_shipped_by_company = orders[orders['ShippedDateKey'] == 1011900]\
    .merge(customers[['CustomerID','Company']], on='CustomerID')\
    .groupby('Company').size().reset_index(name='NotShippedCount')
not_shipped_by_company.to_csv("../Data/analytics/not_shipped_by_company.csv", index=False)



shipped_orders = orders[orders['ShippedDateKey'] != 1011900]

# Merge with DimTime on ShippedDateKey
shipped_orders = shipped_orders.merge(
    time_dim[['DateKey', 'Year', 'Month']], 
    left_on='ShippedDateKey', 
    right_on='DateKey', 
    how='left'
)

shipped_per_year = shipped_orders.groupby('Year').size().reset_index(name='ShippedCount')
shipped_per_year = shipped_per_year.sort_values(by='ShippedCount', ascending=False)
shipped_per_year.to_csv("../Data/analytics/most_shipped_per_year.csv", index=False)

# Most shipped per month (across all years)
shipped_per_month = shipped_orders.groupby('Month').size().reset_index(name='ShippedCount')
shipped_per_month = shipped_per_month.sort_values(by='ShippedCount', ascending=False)
shipped_per_month.to_csv("../Data/analytics/most_shipped_per_month.csv", index=False)


# Regenerate Analytics Files

Run this cell to update all analytics CSV files with the corrected data:

In [ ]:
import pandas as pd

# Reload warehouse data
customers = pd.read_csv("../Data/warehouse/DimCustomer.csv")
employees = pd.read_csv("../Data/warehouse/DimEmployee.csv")
orders = pd.read_csv("../Data/warehouse/Order_Fact.csv")
time_dim = pd.read_csv("../Data/warehouse/DimTime.csv")

# Strip column names
customers.columns = customers.columns.str.strip()
employees.columns = employees.columns.str.strip()
orders.columns = orders.columns.str.strip()
time_dim.columns = time_dim.columns.str.strip()

orders['ShippedDateKey'] = orders['ShippedDateKey'].astype(int)

# Regenerate all analytics files
print("Regenerating analytics files with both Access and SQL Server data...")

# Shipped summary
shipped_summary = pd.DataFrame({
    'ShippedStatus': ['Shipped', 'NotShipped'],
    'Count': [
        (orders['ShippedDateKey'] != 1011900).sum(),
        (orders['ShippedDateKey'] == 1011900).sum()
    ]
})
shipped_summary.to_csv("../Data/analytics/shipped_summary.csv", index=False)
print(f"✓ shipped_summary.csv: {len(shipped_summary)} rows")

# Shipped by employee
shipped_by_employee = orders[orders['ShippedDateKey'] != 1011900]\
    .groupby('EmployeeID').size().reset_index(name='ShippedCount')
shipped_by_employee = shipped_by_employee.merge(employees[['EmployeeID','FullName']], on='EmployeeID')
shipped_by_employee.to_csv("../Data/analytics/shipped_by_employee.csv", index=False)
print(f"✓ shipped_by_employee.csv: {len(shipped_by_employee)} rows")

# Shipped by customer
shipped_by_customer = orders[orders['ShippedDateKey'] != 1011900]\
    .groupby('CustomerID').size().reset_index(name='ShippedCount')
shipped_by_customer = shipped_by_customer.merge(customers[['CustomerID','Company']], on='CustomerID')
shipped_by_customer.to_csv("../Data/analytics/shipped_by_customer.csv", index=False)
print(f"✓ shipped_by_customer.csv: {len(shipped_by_customer)} rows")

# Shipped by country
shipped_by_country = orders[orders['ShippedDateKey'] != 1011900]\
    .merge(customers[['CustomerID','Country']], on='CustomerID')\
    .groupby('Country').size().reset_index(name='ShippedCount')
shipped_by_country.to_csv("../Data/analytics/shipped_by_country.csv", index=False)
print(f"✓ shipped_by_country.csv: {len(shipped_by_country)} rows")

# Shipped by company
shipped_by_company = orders[orders['ShippedDateKey'] != 1011900]\
    .merge(customers[['CustomerID','Company']], on='CustomerID')\
    .groupby('Company').size().reset_index(name='ShippedCount')
shipped_by_company.to_csv("../Data/analytics/shipped_by_company.csv", index=False)
print(f"✓ shipped_by_company.csv: {len(shipped_by_company)} rows")

# Not shipped by employee
not_shipped_by_employee = orders[orders['ShippedDateKey'] == 1011900]\
    .groupby('EmployeeID').size().reset_index(name='NotShippedCount')
not_shipped_by_employee = not_shipped_by_employee.merge(employees[['EmployeeID','FullName']], on='EmployeeID')
not_shipped_by_employee.to_csv("../Data/analytics/not_shipped_by_employee.csv", index=False)
print(f"✓ not_shipped_by_employee.csv: {len(not_shipped_by_employee)} rows")

# Not shipped by customer
not_shipped_by_customer = orders[orders['ShippedDateKey'] == 1011900]\
    .groupby('CustomerID').size().reset_index(name='NotShippedCount')
not_shipped_by_customer = not_shipped_by_customer.merge(customers[['CustomerID','Company']], on='CustomerID')
not_shipped_by_customer.to_csv("../Data/analytics/not_shipped_by_customer.csv", index=False)
print(f"✓ not_shipped_by_customer.csv: {len(not_shipped_by_customer)} rows")

# Not shipped by country
not_shipped_by_country = orders[orders['ShippedDateKey'] == 1011900]\
    .merge(customers[['CustomerID','Country']], on='CustomerID')\
    .groupby('Country').size().reset_index(name='NotShippedCount')
not_shipped_by_country.to_csv("../Data/analytics/not_shipped_by_country.csv", index=False)
print(f"✓ not_shipped_by_country.csv: {len(not_shipped_by_country)} rows")

# Not shipped by company
not_shipped_by_company = orders[orders['ShippedDateKey'] == 1011900]\
    .merge(customers[['CustomerID','Company']], on='CustomerID')\
    .groupby('Company').size().reset_index(name='NotShippedCount')
not_shipped_by_company.to_csv("../Data/analytics/not_shipped_by_company.csv", index=False)
print(f"✓ not_shipped_by_company.csv: {len(not_shipped_by_company)} rows")
print(f"\nSample not shipped companies:\n{not_shipped_by_company.head(10)}")

# Time-based analytics
shipped_orders = orders[orders['ShippedDateKey'] != 1011900]
shipped_orders = shipped_orders.merge(
    time_dim[['DateKey', 'Year', 'Month']], 
    left_on='ShippedDateKey', 
    right_on='DateKey', 
    how='left'
)

shipped_per_year = shipped_orders.groupby('Year').size().reset_index(name='ShippedCount')
shipped_per_year = shipped_per_year.sort_values(by='ShippedCount', ascending=False)
shipped_per_year.to_csv("../Data/analytics/most_shipped_per_year.csv", index=False)
print(f"✓ most_shipped_per_year.csv: {len(shipped_per_year)} rows")

shipped_per_month = shipped_orders.groupby('Month').size().reset_index(name='ShippedCount')
shipped_per_month = shipped_per_month.sort_values(by='ShippedCount', ascending=False)
shipped_per_month.to_csv("../Data/analytics/most_shipped_per_month.csv", index=False)
print(f"✓ most_shipped_per_month.csv: {len(shipped_per_month)} rows")

print("\n✅ All analytics files regenerated successfully!")

# 📊 Projet Business Intelligence : Analyse Northwind Data Warehouse

Ce Notebook Jupyter utilise les données chargées dans le Data Warehouse `NorthwindDW` (via le script `etl.py`) pour générer les analyses clés.

**Outils utilisés :**
* **Connexion :** `pyodbc`
* **Manipulation :** `pandas`
* **Visualisation :** `matplotlib` et `seaborn`

---
## 1. Initialisation et Connexion

La première étape consiste à importer les bibliothèques nécessaires et à établir une connexion sécurisée à l'instance SQL Server.

In [ ]:
import pandas as pd
import pyodbc
import matplotlib.pyplot as plt
import seaborn as sns

# Configuration de la connexion au Data Warehouse
# (Utilise les paramètres identifiés dans le script ETL)
SQL_DW_SERVER = r'DESKTOP-F8N2M8C\SQLEXPRESS'
SQL_DW_DATABASE = 'NorthwindDW'
SQL_DW_DRIVER = '{ODBC Driver 17 for SQL Server}'

# Chaîne de connexion pour l'authentification Windows
SQL_CONN_STRING = (
    f'DRIVER={SQL_DW_DRIVER};'
    f'SERVER={SQL_DW_SERVER};'
    f'DATABASE={SQL_DW_DATABASE};'
    r'Trusted_Connection=yes;'
    r'TrustServerCertificate=yes;'
)

print(f"Tentative de connexion au Data Warehouse: {SQL_DW_DATABASE}...")
try:
    conn = pyodbc.connect(SQL_CONN_STRING)
    print("✅ Connexion réussie. Le Data Warehouse est prêt pour l'analyse.")
except pyodbc.Error as e:
    print(f"❌ Échec de la connexion. Vérifiez le serveur et le pilote : {e}")
    conn = None

---
## 2. Analyse 1 : Tendance des Ventes sur la Période

**Objectif :** Évaluer la performance globale des ventes (`SalesAmount`) mois par mois afin d'identifier toute croissance, stagnation ou déclin.

### **Conclusion :**
Le graphique linéaire révèle une **tendance générale à la baisse** des revenus sur la période. Le pic initial est suivi d'une érosion progressive, signalant une saturation possible du marché ou un besoin de réévaluer les stratégies commerciales.

In [ ]:
if conn:
    # Requête: Tendance des ventes mensuelles (utilise FactSales et DimDate)
    query_sales_trend = """
    SELECT 
        DD.Year,
        DD.Month,
        SUM(FS.SalesAmount) AS MonthlySales
    FROM 
        FactSales FS
    JOIN 
        DimDate DD ON FS.OrderDateKey = DD.DateKey
    GROUP BY 
        DD.Year,
        DD.Month
    ORDER BY 
        DD.Year,
        DD.Month;
    """

    df_sales_trend = pd.read_sql(query_sales_trend, conn)
    
    # Création de la colonne de temps pour l'axe X (Année-Mois)
    df_sales_trend['YearMonth'] = df_sales_trend['Year'].astype(str) + '-' + df_sales_trend['Month'].astype(str).str.zfill(2)

    # Visualisation
    plt.figure(figsize=(12, 6))
    sns.lineplot(x='YearMonth', y='MonthlySales', data=df_sales_trend, marker='o')
    plt.title('Tendance des Ventes Mensuelles (SalesAmount)', fontsize=16)
    plt.xlabel('Mois', fontsize=12)
    plt.ylabel('Revenus Totaux ($)', fontsize=12)
    plt.xticks(rotation=45)
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.show()

---
## 3. Analyse 2 : Performance Commerciale

**Objectif :** Identifier les employés ayant généré le plus de revenus pour récompenser les meilleurs performeurs et analyser leurs méthodes de vente.

### **Conclusion :**
Le classement des ventes montre une nette domination de certains employés, notamment **Margaret Peacock** et **Janet Leverling**. La direction devrait examiner de près leurs zones géographiques et leurs méthodes pour reproduire ce succès.

In [ ]:
if conn:
    # Requête SQL pour agréger les ventes par employé
    sql_query_top_employees = """
    SELECT
        de.FirstName + ' ' + de.LastName AS EmployeeName,
        SUM(fs.SalesAmount) AS TotalSales
    FROM FactSales fs
    JOIN DimEmployees de ON fs.EmployeeID = de.EmployeeKey -- Utilisation de EmployeeID de FactSales et EmployeeKey de DimEmployees
    GROUP BY de.FirstName, de.LastName
    ORDER BY TotalSales DESC;
    """

    df_top_employees = pd.read_sql(sql_query_top_employees, conn)

    # --- Visualisation (Graphique à Barres Horizontal) ---
    plt.figure(figsize=(10, 6))
    sns.barplot(x='TotalSales', y='EmployeeName', data=df_top_employees, palette='viridis')
    plt.title('Performance des Employés (Ventes Totales)')
    plt.xlabel('Montant des Ventes (USD)')
    plt.ylabel('Employé')
    plt.tight_layout()
    plt.show()

    print(f"\nObservation : Top 3 des employés : {df_top_employees.iloc[0]['EmployeeName']}, {df_top_employees.iloc[1]['EmployeeName']}, {df_top_employees.iloc[2]['EmployeeName']}.")

---
## 4. Analyse 3 : Distribution du Volume de Ventes par Catégorie

**Objectif :** Comprendre sur quelles catégories de produits se concentre la majorité des commandes et des articles vendus (`OrderQuantity`).

### **Conclusion :**
Le diagramme circulaire confirme que quelques catégories (typiquement **Seafood** et **Dairy Products**) représentent une part disproportionnée du volume total. Une stratégie marketing et d'approvisionnement devrait se concentrer davantage sur ces segments clés pour maximiser le potentiel de croissance.

In [ ]:
if conn:
    # Requête: Distribution du volume de commandes par Catégorie de Produit
    query_category_volume = """
    SELECT 
        DP.CategoryName,
        SUM(FS.OrderQuantity) AS TotalQuantity
    FROM 
        FactSales FS
    JOIN 
        DimProducts DP ON FS.ProductID = DP.ProductKey
    GROUP BY 
        DP.CategoryName
    ORDER BY 
        TotalQuantity DESC;
    """

    df_category_volume = pd.read_sql(query_category_volume, conn)
    
    # Visualisation (Diagramme Circulaire)
    plt.figure(figsize=(8, 8))
    # Utilisation de autopct='%1.1f%%' pour afficher le pourcentage
    plt.pie(
        df_category_volume['TotalQuantity'], 
        labels=df_category_volume['CategoryName'], 
        autopct='%1.1f%%', 
        startangle=90, 
        wedgeprops={'edgecolor': 'black'}
    )
    plt.title('Distribution du Volume de Commandes par Catégorie', fontsize=16)
    plt.tight_layout()
    plt.show()

In [ ]:
if conn:
    # Requête: Vérification du nombre total de transactions et du CA total
    query_verification = """
    SELECT
        COUNT(DISTINCT OrderID) AS TotalOrders,
        COUNT(SalesAmount) AS TotalOrderDetails,
        SUM(SalesAmount) AS TotalRevenue,
        MIN(OrderDateKey) AS FirstOrderDateKey,
        MAX(OrderDateKey) AS LastOrderDateKey
    FROM
        FactSales;
    """

    df_verification = pd.read_sql(query_verification, conn)
    
    # Affichage des résultats
    print("\n--- Données de FactSales après Consolidation (SQL + Access) ---")
    print(df_verification.to_string(index=False))

In [ ]:
import pandas as pd
from sqlalchemy import create_engine

# 1. Connexion à TON Data Warehouse actuel
engine = create_engine(f'mssql+pyodbc://DESKTOP-F8N2M8C\\SQLEXPRESS/NorthwindDW?driver=ODBC+Driver+17+for+SQL+Server')

# 2. Lecture des tables telles qu'elles sont dans ta base
df_cust = pd.read_sql("SELECT * FROM DimCustomers", engine)
df_emp = pd.read_sql("SELECT * FROM DimEmployees", engine)
df_fact = pd.read_sql("SELECT * FROM FactSales", engine)
df_time = pd.read_sql("SELECT * FROM DimDate", engine)

# 3. TRADUCTION pour correspondre au code de ton ami (On renomme tout ici)
# --- Clients ---
customers = df_cust.rename(columns={
    'CustomerKey': 'CustomerID',
    'CustomerCompanyName': 'Company',
    'CustomerContactName': 'ContactName',
    'CustomerCountry': 'Country'
})

# --- Employés ---
employees = df_emp.copy()
employees['FullName'] = employees['FirstName'] + ' ' + employees['LastName']
employees = employees.rename(columns={'EmployeeKey': 'EmployeeID'})

# --- Temps ---
time_dim = df_time.rename(columns={'DateKey': 'DateKey'})

# --- Faits (La table Orders de ton ami) ---
orders = df_fact.rename(columns={
    'OrderDateKey': 'OrderDateKey',
    'ShippedDateKey': 'ShippedDateKey'
})

# IMPORTANT : Appliquer la logique "1011900" de ton ami pour les non-livrés
# Si la date de livraison est vide (NaN), on met le code 1011900
orders['ShippedDateKey'] = orders['ShippedDateKey'].fillna(1011900).astype(int)

print("✅ Traduction terminée. Les variables 'customers', 'employees', 'orders' et 'time_dim' sont prêtes.")

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# 1. Calcul du résumé des livraisons (Logique 1011900)
shipped_count = (orders['ShippedDateKey'] != 1011900).sum()
not_shipped_count = (orders['ShippedDateKey'] == 1011900).sum()

print(f"📊 Résumé : {shipped_count} commandes livrées / {not_shipped_count} commandes en attente.")

# 2. Performance des employés (Top 5 Livraisons)
shipped_by_emp = orders[orders['ShippedDateKey'] != 1011900].groupby('EmployeeID').size().reset_index(name='Total')
shipped_by_emp = shipped_by_emp.merge(employees[['EmployeeID', 'FullName']], on='EmployeeID')
shipped_by_emp = shipped_by_emp.sort_values('Total', ascending=False)

# 3. Visualisation simple
plt.figure(figsize=(10, 5))
sns.barplot(data=shipped_by_emp.head(5), x='FullName', y='Total', palette='viridis')
plt.title('Top 5 Employés par Commandes Livrées')
plt.xticks(rotation=45)
plt.show()

# 4. Commandes non livrées par Pays (Pour identifier les retards)
not_shipped_geo = orders[orders['ShippedDateKey'] == 1011900].merge(customers[['CustomerID', 'Country']], on='CustomerID')
not_shipped_geo = not_shipped_geo.groupby('Country').size().reset_index(name='EnAttente').sort_values('EnAttente', ascending=False)

print("\n📍 Top pays avec commandes en attente :")
print(not_shipped_geo.head(5).to_string(index=False))

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# 1. Préparation des données pour le camembert (Global)
labels = ['Livré', 'En Attente']
sizes = [shipped_count, not_shipped_count]
colors = ['#66b3ff','#ff9999']

# 2. Performance par Employé (Livré vs Non-Livré)
# On crée une vue pivot pour comparer
emp_perf = orders.groupby(['EmployeeID', orders['ShippedDateKey'] == 1011900]).size().unstack(fill_value=0)
emp_perf.columns = ['Livré', 'En_Attente']
emp_perf = emp_perf.merge(employees[['EmployeeID', 'FullName']], on='EmployeeID')

# --- AFFICHAGE ---
fig = plt.figure(figsize=(15, 10))

# Graphique 1 : Répartition Globale
plt.subplot(2, 2, 1)
plt.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=140, colors=colors, explode=(0.1, 0))
plt.title('État Global des Commandes')

# Graphique 2 : Retards par Pays (Ton TOP 5)
plt.subplot(2, 2, 2)
sns.barplot(data=not_shipped_geo.head(5), x='EnAttente', y='Country', palette='Reds_r')
plt.title('Top 5 des Pays en Retard')

# Graphique 3 : Performance des Employés
plt.subplot(2, 1, 2)
emp_perf_melted = emp_perf.melt(id_vars='FullName', value_vars=['Livré', 'En_Attente'])
sns.barplot(data=emp_perf_melted, x='FullName', y='value', hue='variable', palette={'Livré': 'skyblue', 'En_Attente': 'salmon'})
plt.title('Productivité des Employés : Livraisons vs Retards')
plt.xticks(rotation=30)

plt.tight_layout()
plt.show()

---
## 5. Fin de l'Analyse

La connexion au Data Warehouse a été fermée. Tous les résultats analytiques sont désormais affichés ci-dessus et prêts à être intégrés au Rapport Final de Projet (PDF) et à la vidéo de présentation.

In [ ]:
if conn:
    conn.close()
    print("\nConnexion SQL Server fermée.")